<a href="https://colab.research.google.com/github/fangorntreabeard/nlp-classification/blob/main/ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy

class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, _ = self.lstm(embedded)
        output = output[-1, :, :]
        output = self.fc(output)
        return output

# Предобработка текста и создание словаря
nlp = spacy.load("ru_core_news_sm")

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
    return tokens

def create_vocab(texts):
    vocab = set()
    for text in texts:
        tokens = preprocess_text(text)
        vocab.update(tokens)
    word_to_idx = {word: idx+1 for idx, word in enumerate(vocab)}
    return word_to_idx

# Обучение модели
def train_model(model, train_data, train_labels, num_epochs, batch_size):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        num_batches = len(train_data) // batch_size
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size

            inputs = train_data[start_idx:end_idx]
            labels = train_labels[start_idx:end_idx]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")

# Пример использования
# Подготовка обучающих данных
train_texts = [
    "Этот фильм просто прекрасен!",
    "Эта книга ужасна, не советую читать.",
    "Сегодня погода хорошая."
]
train_labels = torch.tensor([0, 1, 0])

In [31]:
# Подготовка обучающих данных
train_texts = [
    "Этот фильм просто прекрасен!",
    "Эта книга ужасна, не советую читать.",
    "Сегодня погода хорошая."
]
train_labels = torch.tensor([0, 1, 0])  # 0 - положительный, 1 - отрицательный

# Создание словаря
vocab = create_vocab(train_texts)

# Преобразование текста в числовые последовательности
train_data = []
for text in train_texts:
    tokens = preprocess_text(text)
    sequence = [vocab[word] for word in tokens if word in vocab]
    train_data.append(sequence)

# Дополнение последовательностей до одинаковой длины
max_len = max(len(sequence) for sequence in train_data)
train_data = [sequence + [0] * (max_len - len(sequence)) for sequence in train_data]

# Преобразование данных в тензоры PyTorch
train_data = torch.tensor(train_data)
train_labels = train_labels.long()

# Определение гиперпараметров
input_dim = len(vocab) + 1  # +1 для пустого токена
hidden_dim = 100
output_dim = 2  # два класса: положительный и отрицательный
num_epochs = 10
batch_size = 1

# Создание и обучение модели
model = TextClassifier(input_dim, hidden_dim, output_dim)
train_model(model, train_data, train_labels, 11, 4)


Epoch 1/11, Loss: 0.0
Epoch 2/11, Loss: 0.0
Epoch 3/11, Loss: 0.0
Epoch 4/11, Loss: 0.0
Epoch 5/11, Loss: 0.0
Epoch 6/11, Loss: 0.0
Epoch 7/11, Loss: 0.0
Epoch 8/11, Loss: 0.0
Epoch 9/11, Loss: 0.0
Epoch 10/11, Loss: 0.0
Epoch 11/11, Loss: 0.0


In [35]:
# Пример использования обученной модели для предсказания
test_texts = [
    "Этот фильм был отличным!",
    "Мне не понравилась эта книга.",
    "Погода сегодня ужасная."
]

# Предобработка и преобразование тестовых текстов
test_data = []
for text in test_texts:
    tokens = preprocess_text(text)
    sequence = [vocab[word] for word in tokens if word in vocab]
    test_data.append(sequence)

# Дополнение последовательностей до одинаковой длины
test_data = [sequence + [0] * (max_len - len(sequence)) for sequence in test_data]

# Преобразование данных в тензоры PyTorch
test_data = torch.tensor(test_data)

# Вызов предсказания
with torch.no_grad():
    model.eval()
    predictions = model(test_data)

# Преобразование выходов модели в метки классов
predicted_labels = torch.argmax(predictions, dim=1).tolist()

# Вывод результатов
for text, label in zip(test_texts, predicted_labels):
    if label == 0:
        print(f"Текст: {text}\nКлассификация: Положительный\n")
    else:
        print(f"Текст: {text}\nКлассификация: Отрицательный\n")


Текст: Этот фильм был отличным!
Классификация: Положительный

Текст: Мне не понравилась эта книга.
Классификация: Положительный

Текст: Погода сегодня ужасная.
Классификация: Положительный



In [5]:
!pip install spacy[lookups] && python -m spacy download ru_core_news_sm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.3 MB/s eta 0:00:00
2023-06-26 17:20:05.276862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 7.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=407b1e1455172022b8da45b6a281660d87f7e804bd587aea653474ecb628077c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
✔ Download and installation suc

In [103]:
################################################################################
###############################РАБОЧАЯ МОДЕЛЬ###################################
################################################################################

from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)

def preprocess_text(text):
    return text.lower()

train_texts = [
    "Любовь - это сила, которая двигает мир вперед.",
    "Семья - это источник непреходящей поддержки и любви.",
    "Дружба - это связь, которая делает нашу жизнь яркой и значимой.",
    "Деньги - это средство, которое помогает обеспечить нашу жизнь и достижение целей.",
    "Любовь может исцелить и преобразить даже самые тяжелые ситуации.",
    "Семья - это те люди, которые всегда будут рядом, независимо от всего.",
    "Дружба - это взаимопонимание, поддержка и смех на протяжении всей жизни.",
    "Деньги важны, но настоящее богатство заключается в людях, которых мы имеем в своей жизни.",
]

test_texts = [
    "Хотелось бы найти девушку.",
    "Друг -- человек, на которого всегда можно положиться",
    "Финансовые вопросы стали серьезной проблемой для многих людей.",
]

train_labels = [
    [1, 1, 0, 0],  # Любовь, Семья, Дружба, Деньги
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [1, 0, 0, 1],
    [0, 1, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 0, 1],
]
input_ids = []
attention_masks = []
for text in train_texts:
    preprocessed_text = preprocess_text(text)
    encoded_inputs = tokenizer.encode_plus(
        preprocessed_text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded_inputs['input_ids'])
    attention_masks.append(encoded_inputs['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
train_labels = torch.tensor(train_labels, dtype=torch.float32)

# Обучение модели
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask=attention_masks)
    loss = criterion(outputs.logits, train_labels)
    loss.backward()
    optimizer.step()


# Вызов предсказания
with torch.no_grad():
    model.eval()
    test_input_ids = []
    test_attention_masks = []
    for text in test_texts:
        preprocessed_text = preprocess_text(text)
        encoded_inputs = tokenizer.encode_plus(
            preprocessed_text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        test_input_ids.append(encoded_inputs['input_ids'])
        test_attention_masks.append(encoded_inputs['attention_mask'])

    test_input_ids = torch.cat(test_input_ids, dim=0)
    test_attention_masks = torch.cat(test_attention_masks, dim=0)

    outputs = model(test_input_ids, attention_mask=test_attention_masks)
    predicted_labels = torch.sigmoid(outputs.logits)

# Вывод результатов
label_map = {0: "любовь", 1: "семья", 2: "дружба", 3: "деньги"}
for text, labels in zip(test_texts, predicted_labels.tolist()):
    predicted_values = [[label_map[i], label] for i, label in enumerate(labels)]
    print(f"Текст: {text}")
    print(f"Классификация: {predicted_values}\n")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

Текст: Хотелось бы найти девушку.
Классификация: [['любовь', 0.517228901386261], ['семья', 0.39512184262275696], ['дружба', 0.24947544932365417], ['деньги', 0.5338225364685059]]

Текст: Друг -- человек, на которого всегда можно положиться
Классификация: [['любовь', 0.5091134309768677], ['семья', 0.4085809290409088], ['дружба', 0.25483426451683044], ['деньги', 0.5077824592590332]]

Текст: Финансовые вопросы стали серьезной проблемой для многих людей.
Классификация: [['любовь', 0.47441405057907104], ['семья', 0.357241690158844], ['дружба', 0.2906544804573059], ['деньги', 0.5126874446868896]]



In [ ]:
output_dir = "path/to/save/directory"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [37]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00


In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

dataset = [
    {'text': 'Данные 1', 'values': ['любовь', 'дружба']},
    {'text': 'Данные 2', 'values': ['семья', 'деньги']},
]

# Создание словаря ценностей
values = ["любовь", "семья", "дружба", "деньги", ]

# Предобработка текста и создание меток
texts = [data['text'] for data in dataset]
labels = []
for data in dataset:
    label = [0] * len(values)
    for value in data['values']:
        if value in values:
            idx = values.index(value)
            label[idx] = 1
    labels.append(label)

# Разделение на обучающий и тестовый наборы
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Предобработка текста и создание словаря слов
def preprocess_text(text):
    # Ваша логика предобработки текста
    return text.lower()

def create_word_vocab(texts):
    word_to_idx = {}
    for text in texts:
        processed_text = preprocess_text(text)
        for word in processed_text:
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)
    return word_to_idx

# Создание словаря слов
word_to_idx = create_word_vocab(train_texts)

# Преобразование текста в векторы признаков
def text_to_vector(text, word_to_idx):
    processed_text = preprocess_text(text)
    vector = [0] * len(word_to_idx)
    for word in processed_text:
        if word in word_to_idx:
            idx = word_to_idx[word]
            vector[idx] = 1
    return vector

# Преобразование текста обучающего и тестового наборов в векторы признаков
train_vectors = [text_to_vector(text, word_to_idx) for text in train_texts]
test_vectors = [text_to_vector(text, word_to_idx) for text in test_texts]

# Конвертация в тензоры PyTorch
train_data = torch.tensor(train_vectors, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.float32)
test_data = torch.tensor(test_vectors, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.float32)

# Определение модели NLP
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Обучение модели
input_dim = len(word_to_idx)
hidden_dim = 128
output_dim = len(values)

model = TextClassifier(input_dim, hidden_dim, output_dim)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
batch_size = 32

num_batches = len(train_data) / batch_size
for epoch in range(num_epochs):
    epoch_loss = 0.0

    for batch_idx in range(int(num_batches)):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size

        batch_data = train_data[start_idx:end_idx]
        batch_labels = train_labels[start_idx:end_idx]

        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/num_batches}")


# Функция для предсказания
def predict(text, model, word_to_idx, values):
    model.eval()
    with torch.no_grad():
        vector = text_to_vector(text, word_to_idx)
        data = torch.tensor(vector, dtype=torch.float32).unsqueeze(0)
        outputs = model(data)
        probabilities = torch.sigmoid(outputs)
        predicted_labels = (probabilities > 0.5).squeeze().tolist()
        predicted_values = [values[i] for i, label in enumerate(predicted_labels) if label]
        return predicted_values

# Пример использования
example_text = "Сегодня в новостях рассказывают о любви и семье. Также упоминается важность дружбы и денег."
predicted_values = predict(example_text, model, word_to_idx, values)
print(f"Текст: {example_text}")
print("Классификация:", predicted_values)


Epoch 1/1000, Loss: 0.0
Epoch 2/1000, Loss: 0.0
Epoch 3/1000, Loss: 0.0
Epoch 4/1000, Loss: 0.0
Epoch 5/1000, Loss: 0.0
Epoch 6/1000, Loss: 0.0
Epoch 7/1000, Loss: 0.0
Epoch 8/1000, Loss: 0.0
Epoch 9/1000, Loss: 0.0
Epoch 10/1000, Loss: 0.0
Epoch 11/1000, Loss: 0.0
Epoch 12/1000, Loss: 0.0
Epoch 13/1000, Loss: 0.0
Epoch 14/1000, Loss: 0.0
Epoch 15/1000, Loss: 0.0
Epoch 16/1000, Loss: 0.0
Epoch 17/1000, Loss: 0.0
Epoch 18/1000, Loss: 0.0
Epoch 19/1000, Loss: 0.0
Epoch 20/1000, Loss: 0.0
Epoch 21/1000, Loss: 0.0
Epoch 22/1000, Loss: 0.0
Epoch 23/1000, Loss: 0.0
Epoch 24/1000, Loss: 0.0
Epoch 25/1000, Loss: 0.0
Epoch 26/1000, Loss: 0.0
Epoch 27/1000, Loss: 0.0
Epoch 28/1000, Loss: 0.0
Epoch 29/1000, Loss: 0.0
Epoch 30/1000, Loss: 0.0
Epoch 31/1000, Loss: 0.0
Epoch 32/1000, Loss: 0.0
Epoch 33/1000, Loss: 0.0
Epoch 34/1000, Loss: 0.0
Epoch 35/1000, Loss: 0.0
Epoch 36/1000, Loss: 0.0
Epoch 37/1000, Loss: 0.0
Epoch 38/1000, Loss: 0.0
Epoch 39/1000, Loss: 0.0
Epoch 40/1000, Loss: 0.0
Epoch 41/